In [11]:
import numpy as np

# GridWorld et policy_iteration
Le **policy iteration** est un algorithme de programmation dynamique utilisé pour résoudre des problèmes de décision markovienne (MDP). Il s'agit de trouver une politique qui maximise la somme des récompenses attendues sur le long terme pour chaque état du processus. L'algorithme de policy iteration comprend deux phases principales : évaluation de la politique et amélioration de la politique.

In [12]:
class GridWorld:
    def __init__(self, size=5):
        self.S = (size, size)  # grid size
        self.A = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # actions: right, left, down, up
        self.terminal = [(0, 4), (4, 4)]  # terminal states
        self.R = {(0, 4): -3, (4, 4): 1}  # rewards for terminal states

    def is_terminal(self, state):
        return state in self.terminal

    def next_state(self, state, action):
        if self.is_terminal(state):
            return state
        next_state = (state[0] + action[0], state[1] + action[1])
        if 0 <= next_state[0] < self.S[0] and 0 <= next_state[1] < self.S[1]:
            return next_state
        return state  # return the current state if the action leads out of bounds
    
    def reset(self, size=5):
        self.S = (size, size)  # grid size
        self.A = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # actions: right, left, down, up
        self.terminal = [(0, 4), (4, 4)]  # terminal states
        self.R = {(0, 4): -3, (4, 4): 1}  # rewards for terminal states
        
        
def policy_iteration(env, gamma=0.9, theta=0.0001):
    policy = np.zeros((env.S[0], env.S[1], len(env.A))) + 1 / len(env.A)  # Start with a uniform random policy
    stable_policy = False  # Flag to check if the policy has stabilized

    while not stable_policy:
        # Policy Evaluation
        V = np.zeros(env.S)
        while True:
            delta = 0
            for s in range(env.S[0] * env.S[1]):
                x, y = s // env.S[1], s % env.S[1]
                if env.is_terminal((x, y)):
                    continue
                v = V[x, y]
                new_v = 0
                for a, action in enumerate(env.A):
                    next_state = env.next_state((x, y), action)
                    reward = env.R.get(next_state, 0)
                    new_v += policy[x, y, a] * (reward + gamma * V[next_state[0], next_state[1]])
                V[x, y] = new_v
                delta = max(delta, abs(v - new_v))
            if delta < theta:
                break

        # Policy Improvement
        stable_policy = True
        for s in range(env.S[0] * env.S[1]):
            x, y = s // env.S[1], s % env.S[1]
            if env.is_terminal((x, y)):
                continue

            old_action = np.argmax(policy[x, y])
            action_values = np.zeros(len(env.A))
            for a, action in enumerate(env.A):
                next_state = env.next_state((x, y), action)
                reward = env.R.get(next_state, 0)
                action_values[a] = reward + gamma * V[next_state[0], next_state[1]]

            best_action = np.argmax(action_values)
            if old_action != best_action:
                stable_policy = False
            policy[x, y] = np.eye(len(env.A))[best_action]  # Make the policy greedy

    return policy, V


    

In [13]:
env = GridWorld()
#all directions
policy, V = policy_iteration(env)
print('Policy:')
print(policy)


Policy:
[[[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.25 0.25 0.25 0.25]]

 [[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.   0.   1.   0.  ]]

 [[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.   0.   1.   0.  ]]

 [[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.   0.   1.   0.  ]]

 [[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.25 0.25 0.25 0.25]]]


In [14]:
print('Value function:')
print(V)

Value function:
[[0.4782969 0.531441  0.59049   0.6561    0.       ]
 [0.531441  0.59049   0.6561    0.729     0.81     ]
 [0.59049   0.6561    0.729     0.81      0.9      ]
 [0.6561    0.729     0.81      0.9       1.       ]
 [0.729     0.81      0.9       1.        0.       ]]


# Value Iteration dans GridWorld

La Value Iteration est une méthode de programmation dynamique pour trouver la politique optimale dans un processus de décision markovien (MDP). Cette approche est particulièrement efficace pour les environnements avec un espace d'états et d'actions finis, comme dans le cas d'un GridWorld.

### Principe de l'algorithme :

1. **Initialisation** :
   - Initialisez la fonction de valeur V pour tous les états à zéro.

2. **Iteration** :
   - Pour chaque état, calculez la valeur maximale en prenant en compte toutes les actions possibles depuis cet état. Mettez à jour la valeur de l'état avec cette valeur maximale.
   - Répétez ce processus pour tous les états.
   - Continuez cette étape d'itération jusqu'à ce que la différence maximale entre les anciennes valeurs et les nouvelles valeurs des états (delta) soit inférieure à un petit seuil (theta), indiquant la convergence de l'algorithme.

3. **Extraction de la politique** :
   - Une fois la fonction de valeur optimale obtenue, dérivez la politique optimale. Pour chaque état, choisissez l'action qui maximise la somme de la récompense immédiate et la valeur escomptée des états futurs selon la fonction de valeur.

In [15]:
def value_iteration(env, gamma=0.9, theta=0.0001):
    V = np.zeros(env.S)  # Initialize V to zero for all states
    while True:
        delta = 0
        for s in range(env.S[0] * env.S[1]):
            x, y = s // env.S[1], s % env.S[1]
            if env.is_terminal((x, y)):
                continue
            v = V[x, y]
            max_value = float('-inf')
            for action in env.A:
                next_state = env.next_state((x, y), action)
                reward = env.R.get(next_state, 0)
                value = reward + gamma * V[next_state[0], next_state[1]]
                if value > max_value:
                    max_value = value
            V[x, y] = max_value
            delta = max(delta, abs(v - max_value))
        if delta < theta:
            break
    
    # Derive policy from value function
    policy = np.zeros((env.S[0], env.S[1], len(env.A)))
    for s in range(env.S[0] * env.S[1]):
        x, y = s // env.S[1], s % env.S[1]
        if env.is_terminal((x, y)):
            continue
        action_values = np.zeros(len(env.A))
        for a, action in enumerate(env.A):
            next_state = env.next_state((x, y), action)
            reward = env.R.get(next_state, 0)
            action_values[a] = reward + gamma * V[next_state[0], next_state[1]]
        best_action = np.argmax(action_values)
        policy[x, y, best_action] = 1  # Use a deterministic policy

    return policy, V

value_iteration(env)



(array([[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.]],
 
        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],
 
        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],
 
        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],
 
        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.]]]),
 array([[0.4782969, 0.531441 , 0.59049  , 0.6561   , 0.       ],
        [0.531441 , 0.59049  , 0.6561   , 0.729    , 0.81     ],
        [0.59049  , 0.6561   , 0.729    , 0.81     , 0.9      ],
        [0.6561   , 0.729    , 0.81     , 0.9      , 1.       ],
        [0.729    , 0.81     , 0.9      , 1.      